# Generalized Linear Model - Categorical outcome

The linear model can be generalized to include a link function, which allows the modeling of different outcomes besides continuous variables

![glm_overview.png](images/glm_overview.png)

We will explore how we can model a **binary categorical outcome** (e.g. yes/no) using a generalized linear model with a link function log(odds Y)

---
## Data preparation

We will use the dataset from the Framingham study, which was a long term study of cardiovascular disease about subjsects in Framingham, USA. In this dataset ~4000 subjects were followed up and clinical data and chronic heart disease outcomes were examined longitudinally over > 10 years

Variables include:
- `male` : 0 = female, 1 = male
- `age` : Age at the time of medical examination in years.
- `education` : 1 = Some high school, 2 = high school/GED, 3 = some college/vocational school, 4 = college
- `currentSmoker`: Current cigarette smoking at the time of examinations
- `cigsPerDay`: Number of cigarettes smoked each day
- `BPmeds`: Use of Anti-hypertensive medication at exam
- `prevalentStroke`: Prevalent Stroke 
- `prevalentHyp`: Prevalent Hypertensive
- `diabetes`: Diabetic according to criteria of first exam treated
- `totChol`: Total cholesterol (mg/dL)
- `sysBP`: Systolic Blood Pressure (mmHg)
- `diaBP`: Diastolic blood pressure (mmHg)
- `BMI`: Body Mass Index, weight (kg)/height (m)^2
- `heartRate`: Heart rate (beats/minute)
- `glucose`: Blood glucose level (mg/dL)
- `TenYearCHD`: 0 = No Chronic heart disease, 1 = Chronic heart Disease

In [1]:
library(tidyverse)

# load data
data<- read_csv("https://raw.githubusercontent.com/kennethban/dataset/main/framingham.csv")

# rename and change data types
data <- data %>%
        rename(sex = male) %>%
        mutate(sex = as.factor(sex)) %>%
        mutate(education = as.factor(education)) %>%
        mutate(currentSmoker = as.factor(currentSmoker)) %>%
        mutate(BPMeds = as.factor(BPMeds)) %>%
        mutate(prevalentStroke = as.factor(prevalentStroke)) %>%
        mutate(prevalentHyp = as.factor(prevalentHyp)) %>%
        mutate(diabetes = as.factor(diabetes)) %>%
        mutate(TenYearCHD = as.factor(TenYearCHD))

# drop missing values
data <- data %>% drop_na

head(data)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Rows: 4240 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (16): male, age, education, currentSmoker, cigsPerDay, BPMeds, prevalent...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
<fct>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,39,4,0,0,0,0,0,0,195,106.0,70,26.97,80,77,0
0,46,2,0,0,0,0,0,0,250,121.0,81,28.73,95,76,0
1,48,1,1,20,0,0,0,0,245,127.5,80,25.34,75,70,0
0,61,3,1,30,0,0,1,0,225,150.0,95,28.58,65,103,1
0,46,3,1,23,0,0,0,0,285,130.0,84,23.10,85,85,0
0,43,2,0,0,0,0,1,0,228,180.0,110,30.30,77,99,0


We will build a model with the categorical variable `TenYearCHD` as an outcome response and explore its relationship with covariate(s) `BMI` and `age`

---
## Modeling categorical outcomes

Recall in linear regression, we have

$$Y_{i}=\beta_{0}+\beta_{1} X+\varepsilon_{i}$$

In a generalized linear model, we can use a link function

$$f\left(\mu_{Y \mid X}\right)=\beta_{0}+\beta_{1} X$$

For a categorical binary outcome $Y$, we can define

$$
\begin{align}
p(Y) &= \text{probability of }Y\text{ occurring}\\
1-p(Y) &= \text{probability of }Y\text{ not occurring}
\end{align}
$$

Thus,

$$\frac{p(Y)}{1-p(Y)} = \text{odds of }Y\text{ occurring}
$$

We can define a generalized linear equation for a categorical binary outcome $Y$ using a logit function

$$
\log \left(\frac{p(Y)}{1-p(Y)}\right)=\beta_{0}+\beta_{1} X
$$


This is known as logistic regression

## Performing logistic regression with `glm` function

We can perform logistic regression by using the `glm` function
- `outcome variable` ~ `covariate(s)`
- `family = "binomial"` for binary categorical outcomes

In [2]:
model <- glm(TenYearCHD ~ BMI + age, 
              data=data, 
              family="binomial")

model %>% broom::tidy() # tidy output

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-6.54292372,0.407430778,-16.058982,4.945986e-58
BMI,0.03532089,0.011176590,3.160256,1.576305e-03
age,0.07581313,0.005744181,13.198250,8.979749e-40


## How do we interpret the coefficients?

The equation describing the relation can be written as

$$
\log \left(\frac{p(Y)}{1-p(Y)}\right)=-6.54_\text{intercept} + \text{BMI} \times 0.035_\text{BMI} + \text{age} \times 0.076_\text{age} 
$$


Note that the coefficients **do not** have the same linear interpretation as in the linear regression model

We can interpret the coefficients in terms of the link function 

$$\log \left(\frac{p(Y)}{1-p(Y)}\right) = \log(\text{odds})$$

We can consider a covariate at the $n$th term ($b_nx_n$)

Let $x_n = c$

$$
\begin{align}
\log(\text{odds}_{x_n=c}) &= b_0 + \ldots + b_nc\\
\text{odds}_{x_n=c} &= \exp(b_0 + \ldots + b_nc)\\
\end{align}
$$

If we increase $x_n$ by 1, then $x_n = c+1$

$$
\begin{align}
\log(\text{odds}_{x_n=c+1}) &= b_0 + \ldots + b_n(c+1)\\
\text{odds}_{x_n=c+1} &= \exp(b_0 + \ldots + b_n(c+1)\\
\end{align}
$$

We can then find the odds ratio when $x_n$ increases by 1 unit

$$
\begin{align}
\mathit{OR} = \frac{\text{odds}_{x_n=c+1}}{\text{odds}_{x_n=c}} &= \frac{\exp(b_0 + \ldots + b_n(c+1))}{\exp(b_0 + \ldots + b_nc)}\\
&= \exp(b_n)
\end{align}
$$

## Calculating the odds ratio from the coefficients
In this example
- BMI coefficient = 0.035

$$\mathit{OR}_\text{BMI} = \exp(0.035) = 1.036$$

- age coefficient = 0.076

$$\mathit{OR}_\text{age} = \exp(0.076) = 1.079$$

We can see that the $\mathit{OR}$ for both coefficients are >1, indicating that they are associated with a higher odds of developing chronic heart disease after a 10 year follow-up

---
## Application: Statistical table and plotting

In [ ]:
library(tidyverse)

# load data
data<- read_csv("https://raw.githubusercontent.com/kennethban/dataset/main/framingham.csv")

# rename and change data types
data <- data %>%
        rename(sex = male) %>%
        mutate(sex = as.factor(sex)) %>%
        mutate(education = as.factor(education)) %>%
        mutate(currentSmoker = as.factor(currentSmoker)) %>%
        mutate(BPMeds = as.factor(BPMeds)) %>%
        mutate(prevalentStroke = as.factor(prevalentStroke)) %>%
        mutate(prevalentHyp = as.factor(prevalentHyp)) %>%
        mutate(diabetes = as.factor(diabetes)) %>%
        mutate(TenYearCHD = as.factor(TenYearCHD))

# drop missing values
data <- data %>% drop_na

# fit logistic model
model <- glm(TenYearCHD ~ BMI + age, 
              data=data, 
              family="binomial")

### 1. Statistical table

We will use the `stargazer` function from the `stargazer` library to generate a statistical table from the model. The function can take one or more models as its input and we can specify the following options
- `ci`: set to TRUE to print the confidence intervals
- `type`: set to HTML to print the table in the notebook

We will define a local function `print_html` to print the HTML output from `stargazer` in the notebook

In [ ]:
print_html <- function(func) {
    
    capture.output(func) %>% paste(collapse="") %>% IRdisplay::display_html()
    
}

In [ ]:
library(stargazer)

stargazer(model, ci = T, type="html") %>% print_html

Note that the coefficients are reported as log values, so they need to be exponentiated to get the odds ratio $\mathit{OR}$

### 2. Statistical plot

We will use the `ggcoefstats` function from `ggstatsplot` to generate a plot of the coefficients with the confidence intervals and the associate p-values. We can specify the following options
- `exclude.intercept`: set to TRUE to omit the intercept
- `stats.label.args`: provide a list of options for the labels if desired

In [ ]:
library(ggstatsplot)

# adjust size of the image output
options(repr.plot.width=10, repr.plot.height=10)

model %>% ggcoefstats(exclude.intercept = T,
                      stats.label.args=list(nudge_y=0.2, 
                                label.size=NA)) +
          theme_grey(base_size=16)